#### Load Python Files + Extract Comments

In [ ]:
# 📦 Step 1: Imports and Setup
import os
from openai import OpenAI
import json

client = OpenAI(api_key="your_api_key_here")
# File paths
TARGET_FILE = "../data/3-comments-nlp/benchmark_comments.py"
OUTPUT_FILE = "nlp-results.json"

In [16]:
# 🧠 Step 2: Prompt Template Builder
def build_prompt(comment, code_context):
    return f"""
You are a senior code reviewer focused on comment clarity and effectiveness.
Given the following Python comment and its surrounding code, evaluate the comment quality.

Code:
{code_context}

Comment:
{comment}

Tasks:
1. Classify the comment as 'good' or 'bad'
2. Explain why it's good or bad
3. Provide a minimalist suggestion (not the solution, just how to improve it)

Return your answer in this format:
classification: <good|bad>
why: <reason>
suggestion: <1-sentence minimalist improvement>
"""

In [18]:
# 🔍 Step 3: Read the File and Extract Comments
with open(TARGET_FILE, "r") as f:
    lines = f.readlines()

comment_lines = []
window_size = 5

for idx, line in enumerate(lines):
    stripped = line.strip()
    if stripped.startswith("#") and not stripped.startswith("###"):
        comment = stripped
        start = max(0, idx - 2)
        end = min(len(lines), idx + 3)
        context = "".join(lines[start:end])
        comment_lines.append((idx + 1, comment, context))

print(f"Found {len(comment_lines)} comments to analyze.")

Found 10 comments to analyze.


In [22]:
# Step 4: Send to OpenAI and Collect Results
results = []

for line_num, comment, context in comment_lines:
    prompt = build_prompt(comment, context)

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
        )
        # use attribute access on the ChatCompletionMessage object
        content = response.choices[0].message.content
        lines_out = content.strip().splitlines()

        classification = lines_out[0].split(":", 1)[1].strip()
        reason = lines_out[1].split(":", 1)[1].strip()
        suggestion = lines_out[2].split(":", 1)[1].strip()

        if classification == "bad":
            results.append({
                "file": TARGET_FILE,
                "line": line_num,
                "comment": comment,
                "quality": classification,
                "why": reason,
                "suggestion": suggestion
            })
            print(f"✅ Line {line_num}: {classification} → {suggestion}")
        else:
            print(f"ℹ️  Line {line_num}: classified as good, skipping.")

    except Exception as e:
        print(f"❌ Error on line {line_num}: {e}")

ℹ️  Line 3: classified as good, skipping.
✅ Line 9: bad → Provide a brief description of the specific action or purpose of the code following the comment.
✅ Line 14: bad → Add a brief description of what the loop is doing, such as "Doubling each element in the data list."
✅ Line 18: bad → Provide a more descriptive comment such as "Return the data for verification."
✅ Line 24: bad → Add a brief description of the purpose or method used to hash the password, such as "Using SHA-256 for password hashing".
✅ Line 30: bad → Add a more descriptive comment explaining how the user input is being validated.
✅ Line 38: bad → Add a brief description of what kind of data is being sent or how it is being sent to Kafka.
✅ Line 44: bad → Add a brief description of what is being retrieved or why the 'get' operation is being performed.
✅ Line 52: bad → Add a brief explanation of why the variable is being initialized or what its purpose is in the code.
✅ Line 55: bad → Specify the sorting algorithm used

In [ ]:
# 💾 Step 5: Save to JSON
with open(OUTPUT_FILE, "w") as f:
    json.dump(results, f, indent=2)

print(f"📄 NLP results saved to {OUTPUT_FILE}")

📄 NLP results saved to nlp-results.json
